# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,21.91,79,90,4.52,PN,1715130879
1,1,waitangi,-43.9535,-176.5597,11.01,71,9,2.68,NZ,1715130880
2,2,port elizabeth,-33.9180,25.5701,19.50,88,0,10.80,ZA,1715130880
3,3,asaita,11.5636,41.4394,30.37,57,23,1.89,ET,1715130880
4,4,manokwari,-0.8667,134.0833,28.21,76,100,0.36,ID,1715130880


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",                  # xvalues
    "Lat",                  # yvalues
    geo = True,             # set plot to be treated as geographic
    tiles = "OSM",          # OSM tileset
    frame_width = 800,      # how wide frame is 
    frame_height = 600,     # How Tall frame is
    size = "Humidity",      # size of dot is relative to humidity value
    scale = 0.5,            # scaling factor for dot size
    color = "City"          # color is detemined by "City" field
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# Set upper and lower bounds for ideal weather conditions for dream vacation
TempUpperBound = 28
TempLowerBound = 24
WindSpeedUpperBound = 5
CloudinessUpperBound = 25
HumidityUpperBound = 65

#create dataframe filtered from dataset pulled from CSV file
picky_city_data = city_data_df[(city_data_df["Max Temp"] < TempUpperBound) &
                               (city_data_df["Max Temp"] > TempLowerBound) &
                               (city_data_df["Humidity"] < HumidityUpperBound) &
                               (city_data_df["Wind Speed"] < WindSpeedUpperBound) &
                               (city_data_df["Cloudiness"] < CloudinessUpperBound)]

# Drop any rows with null values
picky_city_data.dropna(axis=0,how="any")

# Display sample data
print(len(picky_city_data))

9


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = picky_city_data.copy() #copies dataframe to new dataframe

# grabs all rows from only selected columns
hotel_df = hotel_df.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']]   

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,najran,SA,17.4924,44.1277,40,
119,robore,BO,-18.3333,-59.7500,64,
141,san luis de la loma,MX,17.2706,-100.8939,60,
206,al hazm,YE,16.1641,44.7769,46,
264,mulayjah,SA,27.2710,48.4242,38,
278,tondano,ID,1.3038,124.9112,61,
417,karratha,AU,-20.7377,116.8463,49,
426,farah,AF,32.5000,62.5000,31,
434,sawakin,SD,19.1059,37.3321,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000    # in meters
params = {
    "categories":"accommodation.hotel",    # gives condition for what kind of search we are running
    "limit": 20,                           # serach for the first 20, we will keep closest
    "apiKey":geoapify_key                  # pull apikey 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude =  row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = (f"circle:{longitude},{latitude},{radius}")
    params["bias"] = (f"proximity:{longitude},{latitude}")
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
najran - nearest hotel: No hotel found
robore - nearest hotel: Lajas
san luis de la loma - nearest hotel: No hotel found
al hazm - nearest hotel: No hotel found
mulayjah - nearest hotel: No hotel found
tondano - nearest hotel: Hotel Tou Dano
karratha - nearest hotel: Karratha International Hotel
farah - nearest hotel: No hotel found
sawakin - nearest hotel: Hotel Halfa


,City,Country,Lat,Lng,Humidity,Hotel Name
48,najran,SA,17.4924,44.1277,40,No hotel found
119,robore,BO,-18.3333,-59.7500,64,Lajas
141,san luis de la loma,MX,17.2706,-100.8939,60,No hotel found
206,al hazm,YE,16.1641,44.7769,46,No hotel found
264,mulayjah,SA,27.2710,48.4242,38,No hotel found
278,tondano,ID,1.3038,124.9112,61,Hotel Tou Dano
417,karratha,AU,-20.7377,116.8463,49,Karratha International Hotel
426,farah,AF,32.5000,62.5000,31,No hotel found
434,sawakin,SD,19.1059,37.3321,62,Hotel Halfa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",             # x values
    "Lat",             # y values
    geo = True,        # confirms this is a geographic plot
    tiles = "OSM",     # using OSM tile set
    frame_width = 800, # how wide map is
    frame_height = 300,# how tall map is
    size = "Humidity", # size of dot is relative to humidity value
    color = "City",    # color of dot is determined by city
    hover_cols=["Hotel Name", "Country"]  # adds hotel name and country to hover pop up
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)